In [11]:
import os
import time
import glob
import shutil
import tempfile
from azureml.core import Workspace, Experiment, Run

from auth import get_auth
from src.qa_config import MODEL_CONFIG, EVAL_CONFIG, DATA_CONFIG, RESULT_CONFIG

In [12]:
code_dir = "src"
paths = glob.glob(os.path.join(code_dir, "*.py"))
paths

['src/constants.py', 'src/evaluate.py', 'src/qa_config.py', 'src/utils.py']

In [13]:
#create a temp folder and copy code, model and dataset

In [14]:
print("Creating temp folder...")
temp_path = "tmp_eval"
if os.path.exists(temp_path):
    shutil.rmtree(temp_path)
os.mkdir(temp_path)

for p in paths:
    shutil.copy(p, temp_path)
print("Done.")

Creating temp folder...
Done.


In [15]:
def download_model(ws, EXPERIMENT_NAME, RUN_ID, INPUT_LOCATION, OUTPUT_LOCATION):
    '''
    Download the pretrained model
    Input:
         ws: workspace to access the experiment
         EXPERIMENT_NAME: Name of the experiment in which model is saved
         RUN_ID: Run Id of the experiment in which model is pre-trained 
         INPUT_LOCATION: Input location in a RUN Id
         OUTPUT_LOCATION: Location for saving the model
    '''
    experiment = Experiment(workspace = ws, name = EXPERIMENT_NAME)
    #Download the model on which evaluation need to be done
    run = Run(experiment, run_id = RUN_ID)
    #run.get_details()
    run.download_file(INPUT_LOCATION, OUTPUT_LOCATION)
    print("Successfully downloaded model")

In [17]:
#When run manually use below
ws = Workspace.from_config()
#When run through pipeline use below
#ws = Workspace.from_config(auth = get_auth())

In [18]:
download_model(ws, MODEL_CONFIG.EXPERIMENT_NAME, MODEL_CONFIG.RUN_ID,
    os.path.join(MODEL_CONFIG.INPUT_LOCATION, MODEL_CONFIG.NAME), temp_path)

Successfully downloaded model


In [19]:
experiment = Experiment(workspace = ws, name = EVAL_CONFIG.EXPERIMENT_NAME)

In [20]:
#Find/create a compute target.

In [21]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Compute cluster exists. Just connect to it.
try:
    compute_target = ComputeTarget(workspace = ws, name = EVAL_CONFIG.CLUSTER_NAME)
    print("Found existing compute target.")

# Compute cluster does not exist. Create one.    
except ComputeTargetException:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size = 'Standard_NC6', 
        max_nodes = 4
    )
    compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)
    
compute_target

Found existing compute target.


AmlCompute(workspace=Workspace.create(name='cgm-ml-prod-we-azml', subscription_id='9b5bbfae-d5d1-4aae-a2ca-75159c0c887d', resource_group='cgm-ml-prod-we-rg'), name=gpu-cluster, id=/subscriptions/9b5bbfae-d5d1-4aae-a2ca-75159c0c887d/resourceGroups/cgm-ml-prod-we-rg/providers/Microsoft.MachineLearningServices/workspaces/cgm-ml-prod-we-azml/computes/gpu-cluster, type=AmlCompute, provisioning_state=Succeeded, location=westeurope, tags=None)

In [22]:
dataset = ws.datasets[DATA_CONFIG.NAME]
dataset

{
  "source": [
    "('omdena_datasets', 'depthmap_evaluation_dataset/**')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "a8ac712b-11d1-42cc-99e3-6e3e03134dd3",
    "name": "anon_depthmap_evaluation",
    "version": 1,
    "description": "An evaluation dataset for depthmaps based scans the same distribution as anon_pcd_evalauation",
    "workspace": "Workspace.create(name='cgm-ml-prod-we-azml', subscription_id='9b5bbfae-d5d1-4aae-a2ca-75159c0c887d', resource_group='cgm-ml-prod-we-rg')"
  }
}

In [23]:
from azureml.core.runconfig import MpiConfiguration
from azureml.train.dnn import TensorFlow
TensorFlow.get_supported_versions()

['1.10', '1.12', '1.13', '2.0', '2.1', '2.2']

In [24]:
#parameters used in the evaluation
script_params = {f"--MODEL_{k}": v for k, v in MODEL_CONFIG.items()}
script_params.update({f"--EVAL_{k}": v for k, v in EVAL_CONFIG.items()})
script_params.update({f"--DATA_{k}": v for k, v in DATA_CONFIG.items()})
script_params.update({f"--RESULT_{k}": v for k, v in RESULT_CONFIG.items()})
script_params

{'--MODEL_EXPERIMENT_NAME': 'q3-depthmap-plaincnn-height-95k-ps',
 '--MODEL_RUN_ID': 'q3-depthmap-plaincnn-height-95k-ps_1604844442_da24a8d9',
 '--MODEL_INPUT_LOCATION': 'outputs',
 '--MODEL_NAME': 'best_model.h5',
 '--EVAL_NAME': 'q3-depthmap-plaincnn-height-95k-ps-run_6',
 '--EVAL_EXPERIMENT_NAME': 'QA-pipeline_ps',
 '--EVAL_CLUSTER_NAME': 'gpu-cluster',
 '--EVAL_DEBUG_RUN': False,
 '--EVAL_DEBUG_NUMBER_OF_SCAN': 50,
 '--EVAL_SPLIT_SEED': 0,
 '--DATA_NAME': 'anon_depthmap_evaluation',
 '--DATA_IMAGE_TARGET_HEIGHT': 240,
 '--DATA_IMAGE_TARGET_WIDTH': 180,
 '--DATA_BATCH_SIZE': 256,
 '--DATA_NORMALIZATION_VALUE': 7.5,
 '--DATA_TARGET_INDEXES': [0],
 '--DATA_CODE_TO_SCANTYPE': {'100': '_front',
  '101': '_360',
  '102': '_back',
  '200': '_lyingfront',
  '201': '_lyingrot',
  '202': '_lyingback'},
 '--RESULT_ACCURACIES': [0.2, 0.4, 0.6, 1, 1.2, 2.0, 2.5, 3.0, 4.0, 5.0, 6.0],
 '--RESULT_COLUMNS': ['qrcode', 'artifact', 'scantype', 'GT', 'predicted'],
 '--RESULT_SAVE_PATH': 'outputs/resul

In [25]:
start = time.time()

In [26]:
tags= {}
# Specify pip packages here.
pip_packages = [
    "azureml-dataprep[fuse,pandas]",
    "glob2",
    "opencv-python==4.1.2.30",
    "matplotlib",
]

# Create the estimator.
estimator = TensorFlow(
    source_directory=temp_path,
    compute_target=compute_target,
    entry_script="evaluate.py",
    use_gpu=True,
    framework_version="2.2",
    inputs=[dataset.as_named_input("dataset").as_mount()],
    pip_packages=pip_packages,
    script_params=script_params
)

# Set compute target.
estimator.run_config.target = compute_target

# Run the experiment.
run = experiment.submit(estimator, tags=tags)

# Show run.
run

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.
WARNING - If 'arguments' has been provided here and arguments have been specified in 'run_config', 'arguments' provided in ScriptRunConfig initialization will take precedence.
Submitting /mnt/resource/batch/tasks/shared/LS_root/mounts/clusters/pankaja-compins/code/Users/pankaja_us/cgm-ml/src/common/eval/QA/eval-depthmap-height/tmp_eval directory for run. The size of the directory >= 25 MB, so it can take a few minutes.


Experiment,Id,Type,Status,Details Page,Docs Page
QA-pipeline_ps,QA-pipeline_ps_1605117318_13b13340,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
#Check the logs of the current run until is complete
run.wait_for_completion(show_output=True)

In [ ]:
#Print Completed when run is completed
print(run.get_status())

In [23]:
end = time.time()
print("Total time for evaluation experiment: {} sec".format(end-start))

Total time for evaluation experiment: 5.018014669418335


In [ ]:
#Download the evaluation results of the model 
GET_CSV_FROM_EXPERIMENT_PATH = '.'
run.download_file(RESULT_CONFIG.SAVE_PATH, GET_CSV_FROM_EXPERIMENT_PATH)
print("Downloaded the result.csv")

In [ ]:
#Display the evaluation results
from IPython.display import display, HTML
import pandas as pd

result = pd.read_csv('result.csv')
display(HTML(result.to_html()))

In [ ]:
#Delete temp folder
shutil.rmtree(temp_path)